# SWIFT-HEP / GridPP Workshop - April 2025

## Caching

The Dirac Client is introduced here.
Functionally it works the same as the dask.distributed.Client, but allows for persistent caching.

The following cache locations are supported:
- `local`: to set the directory use `file:///path/to/cache`

Caching options in the works;
- `rucio`: to set the directory use `rucio:///path/to/cache`
- `dirac`: to set the directory use `dirac:///path/to/cache`

In [ ]:
from dask_dirac import DiracClient, DiracCluster
from dask.distributed import LocalCluster, Client
import dask.array as da

In [ ]:
# Task structure has changed in newer versions, so for this using 2024.5.0
import dask
dask.__version__

In [ ]:
cluster = LocalCluster(n_workers=1)

In [ ]:
client = DiracClient(cluster, 
                     cache_location="file:///tmp/dask-cache_05022025")
#client = Client(cluster)

In [ ]:
# Check the cache location and show what files are there
print(client.cache_location)
!ls {client.cache_location[7:]} # remove file:// at the beginning

In [ ]:
# Create a Dask DataFrame directly
dask_array = da.ones((1e4, 1), chunks=(1)) + 20231
dask_array

In [ ]:
result = client.compute(dask_array)

In [ ]:
r = result.result()

In [ ]:
# Check the cache location and show what files are there
print(client.cache_location)
!ls {client.cache_location[7:]} # remove file:// at the beginning

## GPU vs CPU

This is an LUX-ZEPLIN analysis which builds a model of multi-scatter-single-ionisation (MSSI) events from simulated events.
This simulated events are from detector components. 
In this analysis, the simulations (ROOT files) are read using `uproot`, and then events are looped over, selecting MSSI events.
The simulated events here have already gone through a pre-processing so only events classified as single-scatter events are considered.

A more detailed step-by-step description of the analysis is as follows:
1. Simulations of detector components are stored as ROOT files.
2. These files are read using `uproot` into `awkward` arrays.
3. A selection is applied to the data to select MSSI events.
4. A normalization is applied to get the expected rate of these events.
5. Something about building the model.


In addition to the above, this analysis also highlights function decorations with numba for CPU and GPU acceleration.

In [ ]:
import awkward as ak
import numpy as np
from dask.distributed import LocalCluster, Client, progress
import glob
import pandas as pd
import uproot as up
import numba
import dask
from numba import cuda
import math

Define the processing

In [ ]:
"""
FV: Wall Radius cut
"""
@numba.njit
def fv_wall_contour(dt):
    c = [-4.44147071e-14, 1.43684777e-10, -1.82739476e-07, 1.02160174e-04, -2.31617857e-02, -2.05932471e+00]
    wall_r2 = c[0]*dt**5 + c[1]*dt**4 + c[2]*dt**3 + c[3]*dt**2 + c[4]*dt + c[5]
    return wall_r2


@cuda.jit
def fv_wall_contour_cuda(dt):
    return -4.44147071e-14 * math.pow(dt, 5) \
            + 1.43684777e-10 * math.pow(dt, 4) \
            -1.82739476e-07 * math.pow(dt, 3) \
                + 1.02160174e-04 * math.pow(dt, 2) \
                    -2.31617857e-02 * dt \
                        -2.05932471e+00

"""
FV: Radial position
"""
n_phi_slices = 12
phi_slices = np.linspace(-np.pi, np.pi, n_phi_slices + 1) + np.pi/4
phi_slices[phi_slices > np.pi] -= 2*np.pi

@numba.njit
def calc_dR_phi(x, y, dt):

    # Calculate event radii and angles, then mask them according to each slice
    R = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)

    dR_phi = 0.0
    # Process each phi slice
    for slice in range(n_phi_slices):
        phi_min, phi_max = phi_slices[slice], phi_slices[slice + 1]
        if (phi >= phi_min) & (phi < phi_max):
            dR_phi = R - perform_poly(dt, slice)

    return dR_phi

@cuda.jit
def calc_dR_phi_cuda(x, y, dt):
    R = math.sqrt(x ** 2 + y ** 2)
    phi = math.atan2(y, x)

    n_phi_slices = 12
    # Now define array on device
    phi_slices = cuda.local.array(13, dtype=numba.float32) 
    phi_slices[0] = -2.35619449
    phi_slices[1] = -1.83259571
    phi_slices[2] = -1.30899694
    phi_slices[3] = -0.78539816
    phi_slices[4] = -0.26179939
    phi_slices[5] = 0.26179939
    phi_slices[6] = 0.78539816
    phi_slices[7] = 1.30899694
    phi_slices[8] = 1.83259571
    phi_slices[9] = 2.35619449
    phi_slices[10] = 2.87979327
    phi_slices[11] = -2.87979327
    phi_slices[12] = -2.35619449

    for slice in range(n_phi_slices):
        phi_min, phi_max = phi_slices[slice], phi_slices[slice + 1]
        if (phi >= phi_min) and (phi < phi_max):
            return R - perform_poly_cuda(dt, slice)   


"""
FV: Poly
"""
coeffs = np.array([[-1.78880746e-13, 4.91268301e-10, -4.96134607e-07, 2.26430932e-04, -4.71792008e-02, 7.33811298e+01],
            [-1.72264463e-13, 4.59149636e-10, -4.59325165e-07, 2.14612376e-04, -4.85599108e-02, 7.35290867e+01],
            [-3.17099156e-14, 7.26336129e-11, -6.99495385e-08, 3.85531008e-05, -1.33386004e-02, 7.18002889e+01],
            [-6.12280314e-14, 1.67968911e-10, -1.83625538e-07, 1.00457608e-04, -2.86728022e-02, 7.22754350e+01],
            [-1.89897962e-14, 1.52777215e-11, -2.79681508e-09, 1.25689887e-05, -1.33093804e-02, 7.17662251e+01],
            [-2.32118621e-14, 7.30043322e-11, -9.40606298e-08, 6.29728588e-05, -2.28150175e-02, 7.22661091e+01],
            [-8.29749194e-14, 2.31096069e-10, -2.47867121e-07, 1.27576029e-04, -3.24702414e-02, 7.26357609e+01],
            [-2.00718008e-13, 5.44135757e-10, -5.59484466e-07, 2.73028553e-04, -6.46879791e-02, 7.45264998e+01],
            [-7.77420021e-14, 1.97357045e-10, -1.90016273e-07, 8.99659454e-05, -2.30169916e-02, 7.25038258e+01],
            [-5.27296334e-14, 1.49415580e-10, -1.58205132e-07, 8.00275441e-05, -2.13559394e-02, 7.23995451e+01],
            [-6.00198219e-14, 1.55333004e-10, -1.60367908e-07, 7.97754165e-05, -1.94435594e-02, 7.22714399e+01],
            [-8.89919309e-14, 2.40830027e-10, -2.57060475e-07, 1.33002951e-04, -3.32969110e-02, 7.28696020e+01]])

@numba.njit
def perform_poly(dt, c):
    poly_results = coeffs[c][0]*dt**5 + coeffs[c][1]*dt**4 + coeffs[c][2]*dt**3 + coeffs[c][3]*dt**2 + coeffs[c][4]*dt + coeffs[c][5]
    return poly_results

# copy coeffs to cuda shared memory
cuda.to_device(coeffs)

@cuda.jit
def perform_poly_cuda(dt, c):

    return (
        coeffs[c][0] * math.pow(dt, 5) + coeffs[c][1] * math.pow(dt, 4) + coeffs[c][2] * math.pow(dt, 3) +
        coeffs[c][3] * math.pow(dt, 2) + coeffs[c][4] * dt + coeffs[c][5]
    )


"""
FV: R Wall cut based on phi position
"""
@numba.njit
def fv_phi_r(x, y, dt):
    dR_phi = calc_dR_phi(x, y, dt)
    contour = fv_wall_contour(dt) - 3  # add cm stand-off
    expandable = (dt > 71) & (dt < 900)

    mask = ((dR_phi < contour) & expandable) | ((dR_phi < contour) & ~expandable)
    return mask & (dR_phi <= 0)



@cuda.jit
def fv_phi_r_cuda(x, y, dt):    
    dR_phi = calc_dR_phi_cuda(x, y, dt)
    contour = fv_wall_contour_cuda(dt) - 3 # add cm stand-off    
    expandable = dt > 71 and dt < 900

    mask = ((dR_phi < contour) and expandable) or ((dR_phi < contour) and not expandable)
    return mask and dR_phi <= 0


"""
FV: Resistor cutout
"""
@numba.njit
def fv_resistor(x, y):
    res1X, res1Y, res1R = -69.8, 3.5, 6
    res2X, res2Y, res2R = -67.5, -14.3, 6
    
    insideRes1 = (x-res1X)*(x-res1X) + (y-res1Y)*(y-res1Y) > res1R**2
    insideRes2 = (x-res2X)*(x-res2X) + (y-res2Y)*(y-res2Y) > res2R**2
    
    return (insideRes1 & insideRes2)

@cuda.jit
def fv_resistor_cuda(x, y):
    res1X, res1Y, res1R = -69.8, 3.5, 6
    res2X, res2Y, res2R = -67.5, -14.3, 6

    insideRes1 = ((x - res1X) ** 2 + (y - res1Y) ** 2) > res1R**2 
    insideRes2 = ((x - res2X) ** 2 + (y - res2Y) ** 2) > res2R**2

    return insideRes1 and insideRes2

"""
FV: Z
"""
@numba.njit
def fv_z(dt):
    return (dt > 71) & (dt < 1030)

@cuda.jit
def fv_z_cuda(dt):
    return (dt > 71) and (dt < 1030)

"""
ROI: S1 and S2
"""
@numba.njit
def roi(s1c, s2, s2c):
    return (s1c > 3.) and (s1c < 80.) and (s2 > 14.5 * 44.5) and (s2c < 10 ** 4.5)


@cuda.jit
def roi_cuda(s1c, s2, s2c):
    return (s1c > 3.) and (s1c < 80.) and (s2 > 14.5 * 44.5) and (s2c < 10 ** 4.5)

In [ ]:
@numba.jit
def process_events(ss_x, ss_y, ss_dt, ss_s1c, ss_s2, ss_s2c, mc_detS1, mc_detS2):
    n_fv, n_roi, n_fv_roi = 0, 0, 0
    n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi = 0, 0, 0, 0

    # Loop over events
    for i in range(len(ss_x)):
        
        # Evaluate cuts
        cut_bool_resistor_fv = fv_resistor(ss_x[i], ss_y[i])
        cut_bool_z = fv_z(ss_dt[i] / 1000)
        cut_bool_r = fv_phi_r(ss_x[i], ss_y[i], ss_dt[i] / 1000)
        fv_cut = cut_bool_resistor_fv and cut_bool_z and cut_bool_r
        roi_cut = roi(ss_s1c[i], ss_s2[i], ss_s2c[i])

        # Examine if MSSI
        nS1, nS2 = 0, 0
        for j in range(len(mc_detS1[i])):
            if mc_detS1[i][j] > 0.:
                nS1 += 1
            if mc_detS2[i][j] > 0.:
                nS2 += 1
        is_mssi = nS1 > nS2
        
        # Now evaluate cuts and count events
        if is_mssi:
            n_mssi += 1
        if fv_cut:
            n_fv += 1
            if is_mssi:
                n_fv_mssi += 1
            if roi_cut:
                n_fv_roi += 1
                if is_mssi:
                    n_fv_roi_mssi += 1
        if roi_cut:
            n_roi += 1
            if is_mssi:
                n_roi_mssi += 1

    return n_fv, n_roi, n_fv_roi, n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi


@cuda.jit
def process_events_cuda(ss_x, ss_y, ss_dt, ss_s1c, ss_s2, ss_s2c, mc_detS1, mc_detS2, 
                        n_fv, n_roi, n_fv_roi, n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi):

    # Set thread
    i = numba.cuda.grid(1)
    # Don't process out of bounds
    if i >= ss_x.shape[0]:
        return

    # Evaluate cuts
    cut_bool_resistor_fv = fv_resistor_cuda(ss_x[i], ss_y[i])
    cut_bool_z = fv_z_cuda(ss_dt[i] / 1000)
    cut_bool_r = fv_phi_r_cuda(ss_x[i], ss_y[i], ss_dt[i] / 1000)
    fv_cut = cut_bool_resistor_fv and cut_bool_z and cut_bool_r
    roi_cut = roi_cuda(ss_s1c[i], ss_s2[i], ss_s2c[i])

    # Examine if MSSI
    nS1, nS2 = 0, 0
    for j in range(len(mc_detS1[i])):
        if mc_detS1[i][j] > 0.:
            nS1 += 1
        if mc_detS2[i][j] > 0.:
            nS2 += 1
    is_mssi = nS1 > nS2

    # Now evaluate cuts and count events
    if roi_cut:
        cuda.atomic.add(n_roi, 0, 1)
        if is_mssi:
            cuda.atomic.add(n_roi_mssi, 0, 1)

    if is_mssi:
        cuda.atomic.add(n_mssi, 0, 1)
    if fv_cut:
        cuda.atomic.add(n_fv, 0, 1)
        if is_mssi:
            cuda.atomic.add(n_fv_mssi, 0, 1)
        if roi_cut:
            cuda.atomic.add(n_fv_roi, 0, 1)
            if is_mssi:
                cuda.atomic.add(n_fv_roi_mssi, 0, 1)

In [ ]:
def process_file(file):
    component = file.split('/SS_skim_')[1][:-5]

    tfile = up.open(file)
    scatters = tfile['Scatters']
    truth = tfile['RQMCTruth']

    # Read arrays
    ss_x = scatters['ss.x_cm'].array()
    ss_y = scatters['ss.y_cm'].array()
    ss_dt = scatters['ss.driftTime_ns'].array()
    ss_s1c = scatters['ss.correctedS1Area_phd'].array()
    ss_s2 = scatters['ss.s2Area_phd'].array()
    ss_s2c = scatters['ss.correctedS2Area_phd'].array()

    mc_detS1 = truth['mcTruthVertices.detectedS1Photons'].array()
    mc_detS2 = truth['mcTruthVertices.detectedS2Photons'].array()

    n_events = len(ss_x)

    result = process_events(ss_x, ss_y, ss_dt, ss_s1c, ss_s2, ss_s2c, mc_detS1, mc_detS2)
    n_fv, n_roi, n_fv_roi, n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi = result

    event_weight = 1e-6 / 200

    return component, n_events, n_fv, n_roi, n_fv_roi, n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi, event_weight

def process_file_cuda(file):
    component = file.split('/SS_skim_')[1][:-5]

    tfile = up.open(file)
    scatters = tfile['Scatters']
    truth = tfile['RQMCTruth']

    # Read arrays and copy to GPU
    ss_x = cuda.to_device(scatters['ss.x_cm'].array())
    ss_y = cuda.to_device(scatters['ss.y_cm'].array())
    ss_dt = cuda.to_device(scatters['ss.driftTime_ns'].array())
    ss_s1c = cuda.to_device(scatters['ss.correctedS1Area_phd'].array())
    ss_s2 = cuda.to_device(scatters['ss.s2Area_phd'].array())
    ss_s2c = cuda.to_device(scatters['ss.correctedS2Area_phd'].array())

    # Need to zero pad jagged arrays
    det_s1 = truth['mcTruthVertices.detectedS1Photons'].array()
    det_s2 = truth['mcTruthVertices.detectedS2Photons'].array()
    pad_size = ak.max(ak.num(det_s1))
    mc_detS1 = cuda.to_device(ak.fill_none(ak.pad_none(det_s1, pad_size), 0))
    mc_detS2 = cuda.to_device(ak.fill_none(ak.pad_none(det_s2, pad_size), 0))

    # Result counters
    n_fv = cuda.to_device(np.zeros(1, dtype=np.int32))
    n_roi = cuda.to_device(np.zeros(1, dtype=np.int32))
    n_fv_roi = cuda.to_device(np.zeros(1, dtype=np.int32))
    n_mssi = cuda.to_device(np.zeros(1, dtype=np.int32))
    n_fv_mssi = cuda.to_device(np.zeros(1, dtype=np.int32)) 
    n_roi_mssi = cuda.to_device(np.zeros(1, dtype=np.int32)) 
    n_fv_roi_mssi = cuda.to_device(np.zeros(1, dtype=np.int32))

    # Launch kernel
    n_events = len(scatters['ss.x_cm'].array())
    block_size = 256
    n_blocks = (n_events + block_size - 1) // block_size

    process_events_cuda[n_blocks, block_size](ss_x, ss_y, ss_dt, ss_s1c, ss_s2, ss_s2c, mc_detS1, mc_detS2,
                                              n_fv, n_roi, n_fv_roi, n_mssi, n_fv_mssi, n_roi_mssi, n_fv_roi_mssi)

    # Copy result back
    cuda.synchronize()
    n_fv_host = n_fv.copy_to_host()[0]
    n_roi_host = n_roi.copy_to_host()[0]
    n_fv_roi_host = n_fv_roi.copy_to_host()[0]
    n_mssi_host = n_mssi.copy_to_host()[0]
    n_fv_mssi_host = n_fv_mssi.copy_to_host()[0]
    n_roi_mssi_host = n_roi_mssi.copy_to_host()[0]
    n_fv_roi_mssi_host = n_fv_roi_mssi.copy_to_host()[0]

    event_weight = 1e-6 / 200

    return component, n_events, n_fv_host, n_roi_host, n_fv_roi_host, n_mssi_host, n_fv_mssi_host, n_roi_mssi_host, n_fv_roi_mssi_host, event_weight

setup dask cluster

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
client

Select the files to be used. 
In this example, the files are stored locally under `/shared/scratch/ak18773/lz/mssi/`. 
Each file is a ROOT file containing the output of an `LZLAMA` simulation (the `NEST` handler); more details can be found in [arvix:2001.09363](https://arxiv.org/abs/2001.09363)

In [ ]:
files = glob.glob("/shared/scratch/ak18773/lz/mssi/*.root")
print(f'N. files: {len(files)}')
files = files[:6]
print(f'N. files to process: {len(files)}')

In [ ]:
delayed_results = [dask.delayed(process_file)(file) for file in files]
futures = client.compute(delayed_results)

# monitor the progress
progress(futures)

In [ ]:
# Once complete, retrieve the results
results = client.gather(futures)
results_df = pd.DataFrame(results, columns=['Source', 'nSS', 'nSS FV', 'nSS ROI', 'nSS FV ROI', 'nMSSI', 'nMSSI FV', 'nMSSI ROI', 'nMSSI FV ROI', 'eventWeight'])
results_df

In [ ]:
delayed_results = [dask.delayed(process_file_cuda)(file) for file in files]
futures = client.compute(delayed_results)
progress(futures)

In [ ]:
results = client.gather(futures)
results_df_cuda = pd.DataFrame(results, columns=['Source', 'nSS', 'nSS FV', 'nSS ROI', 'nSS FV ROI', 'nMSSI', 'nMSSI FV', 'nMSSI ROI', 'nMSSI FV ROI', 'eventWeight'])
results_df_cuda

### Post processing
Now that we have the fraction of events in each region, we can calculate the rates using the known `decays/day`

In [ ]:
rates = {
    "Co60_CalibrationSourceTubes": 4690.57902,
    "Co60_DomePMTs": 3885.410702,
    "K40_BottomTruss": 28927.99798,
    "K40_DomePMTs": 88935.50817,
    "Th232-early_BottomTPCPMTBodies": 38003.65201,
    "Th232-late_BottomTPCPMTBases": 20626.61384,
    "Th232-late_BottomTPCPMTBodies": 51716.2229,
    "Th232-late_ForwardFieldResistors": 77545.76613,
    "Th232-late_HVInnerCone": 363483.6619,
    "U238-late_AnodeGridWires": 4316.423461
}
rates_df = pd.DataFrame(list(rates.items()), columns=["Source", "Rate (Decays/day)"])
rates_df

In [ ]:
# match up where 'Source' is the same in both dataframes, and combine them
df = pd.merge(results_df, rates_df, on='Source')
df

In [ ]:
df['SS/day'] =  df['nSS'] * df['eventWeight'] * df['Rate (Decays/day)']
df['SS/day FV'] = df['nSS FV'] * df['eventWeight'] * df['Rate (Decays/day)']
df['SS/day ROI'] = df['nSS ROI'] * df['eventWeight'] * df['Rate (Decays/day)']
df['SS/day FV ROI'] = df['nSS FV ROI'] * df['eventWeight'] * df['Rate (Decays/day)']
df['MSSI/day'] = df['nMSSI'] * df['eventWeight'] * df['Rate (Decays/day)']
df['MSSI/day FV'] = df['nMSSI FV'] * df['eventWeight'] * df['Rate (Decays/day)']
df['MSSI/day ROI'] = df['nMSSI ROI'] * df['eventWeight'] * df['Rate (Decays/day)']
df['MSSI/day FV ROI'] = df['nMSSI FV ROI'] * df['eventWeight'] * df['Rate (Decays/day)']

In [ ]:
# Calculate the number of events per day from each source
print('Number of SS events expected per day')
all = df['SS/day'].sum()
in_fv = df['SS/day FV'].sum()
in_roi = df['SS/day ROI'].sum()
in_fv_roi = df['SS/day FV ROI'].sum()
print(f'N. SS / day:   {all}')
print(f'In FV / day:     {in_fv}')
print(f'In ROI / day:     {in_roi}')
print(f'N. FV ROI / day: {in_fv_roi}')
print('----------------------------')
print('Number of MSSI events expected per day')
all = df['MSSI/day'].sum()
in_fv = df['MSSI/day FV'].sum()
in_roi = df['MSSI/day ROI'].sum()
in_fv_roi = df['MSSI/day FV ROI'].sum()
in_dataset = in_fv_roi * 220
print(f'N. MSSI / day:   {all}')
print(f'In FV / day:     {in_fv}')
print(f'In ROI / day:     {in_roi}')
print(f'N. FV ROI / day: {in_fv_roi}')
print('----------------------------')
print('Fraction of SS events that are MSSI')
fraction = df['MSSI/day FV ROI'].sum() / df['SS/day FV ROI'].sum()
print(f'fraction: {fraction:.2f}')

### How does processing time compare?

On GPU00...
* 14.6s # numba.njit
* 45+mins # regular Python